In [23]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import Normalizer

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution1D, MaxPooling1D,Activation, Add, Flatten, Softmax
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import h5py
from keras.models import Model
import matplotlib.pyplot as plt

In [2]:
path='Datas'

train = pd.read_csv(os.path.join(path,'mitbih_train.csv'),header=None) 
test = pd.read_csv(os.path.join(path,'mitbih_test.csv'),header=None)

x_train= train.iloc[:,0:187]
y_train= train.iloc[:,187]

x_test= test.iloc[:,0:187]
y_test= test.iloc[:,187]

In [3]:
scaler1= Normalizer().fit(x_train)
x_train = scaler1.transform(x_train)

scaler2= Normalizer().fit(x_test)
x_test = scaler2.transform(x_test)

In [4]:
x_train=np.array(x_train)
x_test=np.array(x_test)

x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
x_train.shape

(87554, 187, 1)

In [5]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train.shape

(87554, 5)

In [15]:
feature,depth=np.shape(x_train[0])

In [24]:
# k.clear_session()

from keras.layers import Input
from keras.layers.convolutional import Conv1D

inp=Input(shape=(feature,depth))
C= Conv1D(filters=32, kernel_size=5, strides=1)(inp)

C11=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(C)
A11=Activation('relu')(C11)
C12=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A11)
S11=Add()([C12,C])
A12=Activation('relu')(S11)
M11=MaxPooling1D(pool_size=5, strides=2)(A12)

C21=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M11)
A21=Activation('relu')(C21)
C22=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A21)
S21=Add()([C22,M11])
A22=Activation('relu')(S21)
M21=MaxPooling1D(pool_size=5, strides=2)(A22)

C31=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M21)
A31=Activation('relu')(C31)
C32=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A31)
S31=Add()([C32,M21])
A32=Activation('relu')(S31)
M31=MaxPooling1D(pool_size=5, strides=2)(A32)

C41=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M31)
A41=Activation('relu')(C41)
C42=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A41)
S41=Add()([C42,M31])
A42=Activation('relu')(S41)
M41=MaxPooling1D(pool_size=5, strides=2)(A42)

C51=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(M41)
A51=Activation('relu')(C51)
C52=Conv1D(filters=32, kernel_size=5, strides=1, padding='same')(A51)
S51=Add()([C52,M41])
A52=Activation('relu')(S51)
M51=MaxPooling1D(pool_size=5, strides=2)(A52)


F1=Flatten()(M51)

D1=Dense(32)(F1)
A6=Activation('relu')(D1)
D2=Dense(32)(A6)

#output layer
D3=Dense(5)(D2)
A7=Softmax()(D3)

model=Model(inputs=inp, outputs=A7)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 187, 1)       0                                            
__________________________________________________________________________________________________
conv1d_26 (Conv1D)              (None, 183, 32)      192         input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 183, 32)      5152        conv1d_26[0][0]                  
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 183, 32)      0           conv1d_27[0][0]                  
__________________________________________________________________________________________________
conv1d_28 

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=100,epochs=1,verbose=1)

Epoch 1/1
87554/87554 [==============================] - 285s 3ms/step - loss: 0.2420 - acc: 0.9321
